In [1]:
import sqlite3
import pandas as pd
import numpy as np

### Basic Query

In [ ]:
SELECT * 
FROM table
LIMIT 10;

In [ ]:
# in SQL lite
conn = sqlite3.connect('nba_db')
cur = conn.cursor()

In [ ]:
cur.execute("""SELECT *,
               FROM players
               LIMIT 10;""")

### LIKE

In [ ]:
SELECT last_name, height
FROM players
WHERE last_name ILIKE 'jam&';